# Keypoint descriptor methods general comparison

The main function, `get_predictions()`, is executed with the different combinations of methods available with default parameters.

The tests are performed with the **QSD1_W2 dataset**.

### Imports

In [ ]:
import os
import cv2
import numpy as np
import tqdm as tqdm
import pickle
import time
from sklearn.metrics import f1_score

from src.utils.images import load_images_from_directory
from src.utils.keypoint_descriptors import *
from src.utils.ml_metrics import mapk

### Constants

In [ ]:
# Image folders
query_dir = "./data/qsd1_w2/"
bbdd_dir = "./data/BBDD"

### Helper functions

In [ ]:
def lowe_ratio_test(knn_matches, ratio_threshold):
    """
    Applies Lowe's ratio test to filter out poor matches from k-nearest neighbors (k-NN) match results.

    Args:
        knn_matches (list of tuples): A list of tuples where each tuple contains two matches (m, n).
            - `m` and `n` are typically objects with a `distance` attribute, representing the 
              distance between matched features.
        ratio_threshold (float): The threshold ratio to determine if a match is good. A smaller 
            ratio is more strict and filters out more matches.

    Returns:
        list: A list of good matches that pass Lowe's ratio test. Each match in the list is from 
        the first element of the tuple `m` in `knn_matches` that satisfies the ratio test.
    """
    good_matches = []
    for match_pair in knn_matches:
        if len(match_pair) >= 2:
            m, n = match_pair[0], match_pair[1]
            if m.distance < ratio_threshold * n.distance:
                good_matches.append(m)
    
    return good_matches


def function_time_count(function, params):

    """
    Measures the execution time of a given function and returns both the 
    time taken and the function's result.

    Args:
        function (callable): The function to be executed.
        params (list): A tuple of parameters to pass to the function.

    Returns:
    - tuple: A tuple containing:
        - total_time (float): The time taken to execute the function, in seconds.
        - results: The output of the executed function.
    """

    start = time.time()
    results = function(*params)
    end = time.time()
    total_time = end-start

    return total_time, results


def get_key_des_multi_image(images_list, method):
    """
    Identifies keypoints and calculates descriptors for each image in
    a list of loaded images using the specified method.
    
    Args:
    - images_list (list of ndarray): list of loaded images
    - method (str): method to use to extract the keypoints and descriptors
    
    Returns:
    - key_des_list (list of dictionaries): list of dictionaries, each
                    dictionary containing the keypoints and descriptors
                    for each image.
    """
    
    if method=="SIFT":
        key_des_list = get_SIFT_key_des_multi_image(images_list)
    
    elif method=="ORB":
        key_des_list = get_ORB_key_des_multi_image(images_list)
        
    elif method=="AKAZE":
        key_des_list = get_AKAZE_key_des_multi_image(images_list)
        
    elif method=="Harris-SIFT":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_key, get_SIFT_descriptors)
    
    elif method=="Harris-ORB":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_key, get_ORB_descriptors)

    elif method=="Harris-AKAZE":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_key, get_AKAZE_descriptors)
    
    elif method=="HarrisLaplacian-SIFT":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_Laplacian_keypoints, get_SIFT_descriptors)
    
    elif method=="HarrisLaplacian-ORB":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_Laplacian_keypoints, get_ORB_descriptors)
    
    elif method=="HarrisLaplacian-AKAZE":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_Laplacian_keypoints, get_AKAZE_descriptors)
    
    return key_des_list

def get_num_matching_descriptors(descriptors_image_1, descriptors_image_2, method, descr_method, params=[]):
    """
    Matches descriptors between two images using either Brute-Force or FLANN-based matching.

    Parameters:
        descriptors_image_1: ndarray
            Descriptors from the first image.
        descriptors_image_2: ndarray
            Descriptors from the second image.
        method: str
            Matching method to use. Options:
            - "BruteForce": Uses Brute-Force matcher.
            - "FLANN": Uses FLANN-based matcher.
        descr_method: str
            Descriptor method used for extracting features. Options:
            - "SIFT": Uses floating-point descriptors.
            - "ORB", "AKAZE": Use binary descriptors.
        params: list, optional
            Additional parameters depending on the method:
            - For "BruteForce":
                params[0]: int
                    Norm type (default: cv2.NORM_L2 for SIFT, cv2.NORM_HAMMING for ORB/AKAZE).
                params[1]: bool
                    Whether to use crossCheck (default = False).
            - For "FLANN":
                params[0]: dict
                    Index parameters.
                params[1]: dict
                    Search parameters.
                params[2]: int
                    Number of nearest neighbors (k) (default: 5).
                params[3]: float
                    Lowe's ratio for filtering matches (default: 0.7).

    Returns:
    - tuple:
        - matches: list
            List of matched descriptors.
        - num_matches: int
            The number of matches found.

    Notes:
    - BruteForce:
        - Uses Euclidean distance for SIFT.
        - Uses Hamming distance for ORB and AKAZE.
    - FLANN:
        - Uses KDTree for SIFT.
        - Uses LSH for ORB and AKAZE.
        - Applies Lowe's ratio test for FLANN-based matches.
    """
    if method == "BruteForce":
        if descr_method in ["SIFT", "Harris-SIFT", "HarrisLaplacian-SIFT"]:
            if params:
                norm = params[0]
                crossCheck = params[1]
            else:
                norm = cv2.NORM_L2
                crossCheck = False

        elif descr_method in ["ORB","AKAZE", "Harris-ORB", "Harris-AKAZE", "HarrisLaplacian-ORB", "HarrisLaplacian-AKAZE"]:
            if params:
                norm = params[0]
                crossCheck = params[1]
            else:
                norm = cv2.NORM_HAMMING
                crossCheck = False
        else:
            norm = cv2.NORM_HAMMING
            crossCheck = True

        matcher = cv2.BFMatcher(norm, crossCheck)
        matches = matcher.match(descriptors_image_1, descriptors_image_2)
        num_matches = len(matches)

    elif method == "FLANN":
        if descr_method == ["SIFT", "Harris-SIFT", "HarrisLaplacian-SIFT"]:
            if params:
                index_params = params[0]
                search_params = params[1]
                k = params[2]
                ratio = params[3]
            else:
                index_params = dict(algorithm=1, trees=5)
                search_params = dict(checks=50)
                k = 2
                ratio = 0.7

        elif descr_method in ["ORB","AKAZE", "Harris-ORB", "Harris-AKAZE", "HarrisLaplacian-ORB", "HarrisLaplacian-AKAZE"]:
            if params:
                index_params = params[0]
                search_params = params[1]
                k = params[2]
                ratio = params[3]
            else:
                index_params = dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1)
                search_params = dict(checks=50)
                k = 2
                ratio = 0.7
        else:
            index_params = dict(algorithm=1, trees=5)
            search_params = dict(checks=50)
            k = 2
            ratio = 0.7
        
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        knn_matches = matcher.knnMatch(descriptors_image_1, descriptors_image_2, k)
        matches = lowe_ratio_test(knn_matches, ratio)
        num_matches = len(matches)

    return matches, num_matches

### Main function

In [ ]:
def get_predictions(query_dir, bbdd_dir, method, matching_method, matching_params=[]):

    # EXTRACT LOCAL FEATURES (KEYPOINT DESCRIPTORS) FROM
    # DENOISED QUERIES AND BBDD
    # =======================================================================================

    # Load denoised query paintings and bbdd images
    query_images = load_images_from_directory(query_dir)
    bbdd_images = load_images_from_directory(bbdd_dir)

    # Extract keypoints and descriptors
    query_key_des_list = get_key_des_multi_image(query_images, method)
    bbdd_key_des_list = get_key_des_multi_image(bbdd_images, method)
    
    
    # GET PREDICTIONS USING MATCHING DESCRIPTORS
    # =======================================================================================
    
    # Results matrix
    results = []
    
    # For each query
    for query_image in tqdm(query_key_des_list, desc="Matching descriptors"):
        
        # Get matching descriptors from each bbdd image
        num_matching_descriptors_list = []
        for bbdd_image in bbdd_key_des_list:
            
            # There must be at least one descriptor in the bbdd image
            if str(bbdd_image['descriptors'])!="None":
                _, num_matching_descriptors = get_num_matching_descriptors(query_image['descriptors'],
                                                                           bbdd_image['descriptors'],
                                                                           method=matching_method,
                                                                           descr_method=method,
                                                                           params=matching_params)
            else:
                num_matching_descriptors = 0
                
            num_matching_descriptors_list.append(num_matching_descriptors)
            
        # Append sorted list of predictions to results list
        results.append(np.argsort(num_matching_descriptors_list)[::-1])
    
    return results

### Loading the groundtruth

In [ ]:
# Load groundtruth
with open('./data/qsd1_w2/gt_corresps.pkl', 'rb') as f:
    ground_truth = pickle.load(f)

### Full results list

In [ ]:
# Store all the results here
full_results = []

### Method testing

#### Methods that execute correctly

In [ ]:
method = "ORB"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, orb_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

orb_info = {}
orb_info['execution_time'] = exec_time
orb_info['MAPK@1'] = mapk(ground_truth, orb_results, k=1)
orb_info['MAPK@5'] = mapk(ground_truth, orb_results, k=5)

full_results.append({"ORB": orb_info})

In [ ]:
method = "AKAZE"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, akaze_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

akaze_info = {}
akaze_info['execution_time'] = exec_time
akaze_info['MAPK@1'] = mapk(ground_truth, akaze_results, k=1)
akaze_info['MAPK@5'] = mapk(ground_truth, akaze_results, k=5)

full_results.append({"AKAZE": akaze_info})

In [ ]:
method = "HarrisLaplacian-ORB"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, hl_orb_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

hl_orb_info = {}
hl_orb_info['execution_time'] = exec_time
hl_orb_info['MAPK@1'] = mapk(ground_truth, hl_orb_results, k=1)
hl_orb_info['MAPK@5'] = mapk(ground_truth, hl_orb_results, k=5)

full_results.append({"HL-ORB": hl_orb_info})

In [ ]:
method = "HarrisLaplacian-ORB"
matching_method = "FLANN"
matching_params = []

exec_time, hl_orb_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

hl_orb_flann_info = {}
hl_orb_flann_info['execution_time'] = exec_time
hl_orb_flann_info['MAPK@1'] = mapk(ground_truth, hl_orb_flann_results, k=1)
hl_orb_flann_info['MAPK@5'] = mapk(ground_truth, hl_orb_flann_results, k=5)

full_results.append({"HL-ORB-FLANN": hl_orb_flann_info})

In [ ]:
method = "SIFT"
matching_method = "FLANN"
matching_params = []

exec_time, sift_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

sift_flann_info = {}
sift_flann_info['execution_time'] = exec_time
sift_flann_info['MAPK@1'] = mapk(ground_truth, sift_flann_results, k=1)
sift_flann_info['MAPK@5'] = mapk(ground_truth, sift_flann_results, k=5)

full_results.append({"SIFT_FLANN": sift_flann_info})

In [ ]:
method = "ORB"
matching_method = "FLANN"
matching_params = []

exec_time, orb_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

orb_flann_info = {}
orb_flann_info['execution_time'] = exec_time
orb_flann_info['MAPK@1'] = mapk(ground_truth, orb_flann_results, k=1)
orb_flann_info['MAPK@5'] = mapk(ground_truth, orb_flann_results, k=5)

full_results.append({"ORB-FLANN": orb_flann_info})

In [ ]:
method = "Harris-ORB"
matching_method = "FLANN"
matching_params = []

exec_time, h_orb_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

h_orb_flann_info = {}
h_orb_flann_info['execution_time'] = exec_time
h_orb_flann_info['MAPK@1'] = mapk(ground_truth, h_orb_flann_results, k=1)
h_orb_flann_info['MAPK@5'] = mapk(ground_truth, h_orb_flann_results, k=5)

full_results.append({"H-ORB-FLANN": h_orb_flann_info})

In [ ]:
method = "Harris-SIFT"
matching_method = "FLANN"
matching_params = []

exec_time, h_sift_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

h_sift_flann_info = {}
h_sift_flann_info['execution_time'] = exec_time
h_sift_flann_info['MAPK@1'] = mapk(ground_truth, h_sift_flann_results, k=1)
h_sift_flann_info['MAPK@5'] = mapk(ground_truth, h_sift_flann_results, k=5)

full_results.append({"H-SIFT-FLANN": h_sift_flann_info})

In [ ]:
method = "HarrisLaplacian-SIFT"
matching_method = "FLANN"
matching_params = []

exec_time, hl_sift_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

hl_sift_flann_info = {}
hl_sift_flann_info['execution_time'] = exec_time
hl_sift_flann_info['MAPK@1'] = mapk(ground_truth, hl_sift_flann_results, k=1)
hl_sift_flann_info['MAPK@5'] = mapk(ground_truth, hl_sift_flann_results, k=5)

full_results.append({"HL-SIFT": hl_sift_flann_info})

In [ ]:
method = "ORB"
matching_method = "FLANN"
matching_params = [dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1), 
                   dict(checks=50), 2, 0.8]

exec_time, orb_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

orb_flann_info = {}
orb_flann_info['execution_time'] = exec_time
orb_flann_info['MAPK@1'] = mapk(ground_truth, orb_flann_results, k=1)
orb_flann_info['MAPK@5'] = mapk(ground_truth, orb_flann_results, k=5)

full_results.append({"ORB-FLANN": orb_flann_info})

#### Methods that don't execute correctly

In [ ]:
method = "Harris-ORB"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, h_orb_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

h_orb_info = {}
h_orb_info['execution_time'] = exec_time
h_orb_info['MAPK@1'] = mapk(ground_truth, h_orb_results, k=1)
h_orb_info['MAPK@5'] = mapk(ground_truth, h_orb_results, k=5)

full_results.append({"H-ORB": h_orb_info})

In [ ]:
method = "Harris-AKAZE"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, h_akaze_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

h_akaze_info = {}
h_akaze_info['execution_time'] = exec_time
h_akaze_info['MAPK@1'] = mapk(ground_truth, h_akaze_results, k=1)
h_akaze_info['MAPK@5'] = mapk(ground_truth, h_akaze_results, k=5)

full_results.append({"H-AKAZE": h_akaze_info})

In [ ]:
method = "AKAZE"
matching_method = "FLANN"
matching_params = []

exec_time, akaze_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

akaze__flann_info = {}
akaze_flann_info['execution_time'] = exec_time
akaze_flann_info['MAPK@1'] = mapk(ground_truth, akaze_flann_results, k=1)
akaze_flann_info['MAPK@5'] = mapk(ground_truth, akaze_flann_results, k=5)

full_results.append({"AKAZE_FLANN": akaze_flann_info})

In [ ]:
method = "Harris-AKAZE"
matching_method = "FLANN"
matching_params = []

exec_time, h_akaze_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

h_akaze_flann_info = {}
h_akaze_flann_info['execution_time'] = exec_time
h_akaze_flann_info['MAPK@1'] = mapk(ground_truth, h_akaze_flann_results, k=1)
h_akaze_flann_info['MAPK@5'] = mapk(ground_truth, h_akaze_flann_results, k=5)

full_results.append({"H-AKAZE-FLANN": h_akaze_flann_info})

In [ ]:
method = "HarrisLaplacian-AKAZE"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, hl_akaze_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

hl_akaze_info = {}
hl_akaze_info['execution_time'] = exec_time
hl_akaze_info['MAPK@1'] = mapk(ground_truth, hl_akaze_results, k=1)
hl_akaze_info['MAPK@5'] = mapk(ground_truth, hl_akaze_results, k=5)

full_results.append({"HL-AKAZE": hl_akaze_info})

In [ ]:
method = "HarrisLaplacian-AKAZE"
matching_method = "FLANN"
matching_params = []

exec_time, hl_akaze_flann_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

In [ ]:
method = "Harris-SIFT"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, h_sift_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

h_sift_info = {}
h_sift_info['execution_time'] = exec_time
h_sift_info['MAPK@1'] = mapk(ground_truth, h_sift_results, k=1)
h_sift_info['MAPK@5'] = mapk(ground_truth, h_sift_results, k=5)

full_results.append({"H-SIFT": h_sift_info})

In [ ]:
method = "HarrisLaplacian-SIFT"
matching_method = "BruteForce"
matching_params = [cv2.NORM_HAMMING, True]

exec_time, hl_sift_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

hl_sift_info = {}
hl_sift_info['execution_time'] = exec_time
hl_sift_info['MAPK@1'] = mapk(ground_truth, hl_sift_results, k=1)
hl_sift_info['MAPK@5'] = mapk(ground_truth, hl_sift_results, k=5)

full_results.append({"HL-SIFT": hl_sift_info})

#### Methods that are too computationally costly

In [ ]:
method = "SIFT"
matching_method = "BruteForce"
matching_params = [cv2.NORM_L2, True]

exec_time, sift_results = function_time_count(get_predictions, (query_dir, bbdd_dir, method, matching_method, matching_params))

sift_info = {}
sift_info['execution_time'] = exec_time
sift_info['MAPK@1'] = mapk(ground_truth, sift_results, k=1)
sift_info['MAPK@5'] = mapk(ground_truth, sift_results, k=5)

full_results.append({"SIFT": sift_info})